In [1]:
import os
import re
import operator
import numpy as np
import string
import nltk


In [3]:
# 遍历出所有文件
all_file = []

root = 'd:/books'

def find_all(root, target):
    for i in os.listdir(root):
        cur = os.path.join(root, i)
        if os.path.isdir(cur):
            find_all(cur, target)
        else:
            target.append(cur)
find_all(root, all_file)

In [220]:
len(all_file)

4574

In [6]:
all_file[:10]

['d:/books\\0\\0_convert - Wei Zhi.pdf',
 'd:/books\\0\\0_ISLR First Printing.pdf',
 'd:/books\\0\\2018-10-29\\designing-data-improving-experience-testing.azw3',
 'd:/books\\0\\2018-10-29\\designing-data-improving-experience-testing.epub',
 'd:/books\\0\\2018-10-29\\designing-data-improving-experience-testing.pdf',
 'd:/books\\0\\2018-10-29\\elegant-scipy-art-scientific-python.azw3',
 'd:/books\\0\\2018-10-29\\elegant-scipy-art-scientific-python.epub',
 'd:/books\\0\\2018-10-29\\elegant-scipy-art-scientific-python.pdf',
 'd:/books\\0\\2018-10-29\\enterprise-java-microservices.pdf',
 'd:/books\\0\\2018-10-29\\handbook-mathematical-formulas-integrals-4th.pdf']

In [7]:
d = {}

for i in all_file:
    file_name = os.path.basename(i)
    tmp = re.split('[-_.\s+]', file_name)
    tmp = [x.lower() for x in tmp]
    for k in tmp[:-1]:
        if len(k)<2:
            continue
        if re.findall('\d+', k):
            continue
        if not k in d:
            d[k] = 0
        d[k] += 1

In [8]:
sort_d = sorted(d.items(), key=operator.itemgetter(1), reverse=True)

In [9]:
sort_d[:10]

[('image', 617),
 ('oreilly', 458),
 ('文字版', 302),
 ('edition', 242),
 ('in', 242),
 ('mask', 235),
 ('the', 206),
 ('learning', 201),
 ('with', 175),
 ('and', 174)]

# 如何找到主题词
文档频率，如果一个词，出现在很多文档中，那么也可以忽略


# 如何处理交叉的情况
如果不能做到正交分解，那么最好有层级结构
最小粒度，不可再分
例如，一个java，一个machine learning，那么machine learning层级更小，因为描述了更具体的事物，熵更小

# 用决策树做分类
需要手动分好训练集

In [63]:
from sklearn import tree
from sklearn import preprocessing

In [61]:
model = tree.DecisionTreeClassifier()

In [62]:
x = ['machine learning in action', 'java cookbook', 'machine learning cookbook', 'hadoop definitive']
y = ['ml', 'java', 'ml', 'hadoop']

In [64]:
encoder = preprocessing.LabelEncoder()

In [66]:
label = encoder.fit_transform(y)

In [67]:
label

array([2, 1, 2, 0], dtype=int64)

In [73]:
# 构建字典
a = [set(i.split(' ')) for i in x]
target = set()
for tmp in a:
    target |= tmp
target = list(target)

In [74]:
target

['java',
 'action',
 'definitive',
 'machine',
 'in',
 'hadoop',
 'learning',
 'cookbook']

In [91]:
# onehot编码
X = []
for tmp in x:
    s = np.zeros(len(target))
    for k in tmp.split(' '):
        s[target.index(k)] = 1
    X.append(s)
X = np.array(X)
print(X)

[[0. 1. 0. 1. 1. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1. 1.]
 [0. 0. 1. 0. 0. 1. 0. 0.]]


In [93]:
# 对于多分类，决策树如何处理深度？
model.fit(X, label)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [94]:
model.predict(X)

array([2, 1, 2, 0], dtype=int64)

In [92]:
label

array([2, 1, 2, 0], dtype=int64)

In [72]:
print(target)

{'java', 'action', 'definitive', 'machine', 'in', 'hadoop', 'learning', 'cookbook'}


In [11]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [12]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ze.liu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ze.liu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [14]:
nltk.word_tokenize()

['d',
 ':',
 '/books\\0\\2018-10-29\\designing-data-improving-experience-testing.azw3']

In [51]:
s = 'd:/books\\0\\2018-10-29\\designing-data-improving-experience-testing.azw3'
s = s.lower()

remove_punc = dict((ord(char), ' ') for char in string.punctuation)

# 用tf-idf计算分数，选最高分

In [182]:
# 只看pdf
filter_file = []
for s in all_file:
    if not s.endswith('pdf'):
        continue
    filter_file.append(s)

In [206]:
# jieba分词
all_doc = []
for s in filter_file:
    s = os.path.basename(s)
    s = s.split('.')[0]
    s = s.lower()
    s = list(jieba.cut(s))
    all_doc.append([x for x in s if len(x)>2 and len(re.findall('\d+', x))==0])
    
# 全量次，做onehot
all_words = set()
for k in all_doc:
    all_words |= set(k)
all_words = list(all_words)

In [207]:
# 向量化
n = len(all_words)
vec = []

for title in all_doc:
    l = np.zeros(n)
    for word in title:
        l[all_words.index(word)] = 1
    vec.append(l)
vec = np.array(vec)

# 计算tf
tf = vec/np.sum(vec)

idf = []

x = np.sum(vec, axis=0)
for i in range(len(all_doc)):
    t = x.copy()
    t[vec[i]<=0] = 0
    idf.append(t)
    
idf = np.array(idf)

# 计算idf
idf = np.log10((len(all_doc)+1)/(idf+1))

tfidf = tf*idf

# 排序
t = np.argsort(tfidf)

all_words = np.array(all_words)

In [201]:
all_words[t[0][-5:]]

array(['leader', 'projectmanagement', 'convert', 'zhi', 'wei'],
      dtype='<U53')

In [204]:
t.shape

(1977, 1379)

In [205]:
len(filter_file)

1977

In [208]:
# 取出分数最高的关键字

for index, d in enumerate(t):
    print('doc', index, all_words[d[-1:]])

doc 0 ['wei']
doc 1 ['islr']
doc 2 ['improving']
doc 3 ['elegant']
doc 4 ['microservices']
doc 5 ['integrals']
doc 6 ['playbook']
doc 7 ['jeffe']
doc 8 ['api']
doc 9 ['deeplearningbook']
doc 10 ['程序设计']
doc 11 ['yearning']
doc 12 ['machine']
doc 13 ['xref']
doc 14 ['mit']
doc 15 ['projectmanagement']
doc 16 ['oreilly']
doc 17 ['oreilly']
doc 18 ['pml']
doc 19 ['pro']
doc 20 ['collective']
doc 21 ['themespotlight']
doc 22 ['tidy']
doc 23 ['whitebook']
doc 24 ['work']
doc 25 ['程序设计']
doc 26 ['程序设计']
doc 27 ['爆发式']
doc 28 ['pdf']
doc 29 ['詹姆斯']
doc 30 ['pattern']
doc 31 ['projectmanagement']
doc 32 ['计算机网络']
doc 33 ['终极版']
doc 34 ['intelligen']
doc 35 ['oreilly']
doc 36 ['activemq']
doc 37 ['metrics']
doc 38 ['clean']
doc 39 ['manning']
doc 40 ['rush']
doc 41 ['ajax']
doc 42 ['effective']
doc 43 ['wayne']
doc 44 ['nutshell']
doc 45 ['algorithms']
doc 46 ['algorithms']
doc 47 ['awp']
doc 48 ['awp']
doc 49 ['introduction']
doc 50 ['introduction']
doc 51 ['machine']
doc 52 ['microsoft']
doc 

doc 579 ['java']
doc 580 ['test']
doc 581 ['java']
doc 582 ['java']
doc 583 ['中文版']
doc 584 ['java']
doc 585 ['数据库']
doc 586 ['数据库']
doc 587 ['java']
doc 588 ['thread']
doc 589 ['完整版']
doc 590 ['manning']
doc 591 ['manning']
doc 592 ['management']
doc 593 ['recipes']
doc 594 ['natural']
doc 595 ['optimizing']
doc 596 ['oreilly']
doc 597 ['oreilly']
doc 598 ['oreilly']
doc 599 ['pragmatic']
doc 600 ['pragmatic']
doc 601 ['pragmatic']
doc 602 ['nio']
doc 603 ['scalable']
doc 604 ['specification']
doc 605 ['virtual']
doc 606 ['think']
doc 607 ['thinking']
doc 608 ['wrox']
doc 609 ['continuous']
doc 610 ['effective']
doc 611 ['functional']
doc 612 ['java']
doc 613 ['java']
doc 614 ['java']
doc 615 ['java']
doc 616 ['java']
doc 617 ['java']
doc 618 ['java']
doc 619 ['java']
doc 620 ['java']
doc 621 ['java']
doc 622 ['java']
doc 623 ['java']
doc 624 ['设计模式']
doc 625 ['killer']
doc 626 ['learning']
doc 627 ['making']
doc 628 ['few']
doc 629 ['oca']
doc 630 ['professional']
doc 631 ['restful']

doc 1079 ['week']
doc 1080 ['deep']
doc 1081 ['keras']
doc 1082 ['practitioners']
doc 1083 ['deep']
doc 1084 ['designing']
doc 1085 ['discrete']
doc 1086 ['doing']
doc 1087 ['encyclopedia']
doc 1088 ['cheatsheet']
doc 1089 ['math']
doc 1090 ['habits']
doc 1091 ['engineering']
doc 1092 ['about']
doc 1093 ['business']
doc 1094 ['engineers']
doc 1095 ['statistical']
doc 1096 ['introductory']
doc 1097 ['jupyter']
doc 1098 ['large']
doc 1099 ['tensorflow']
doc 1100 ['algorithmic']
doc 1101 ['enterprises']
doc 1102 ['constraint']
doc 1103 ['developers']
doc 1104 ['end']
doc 1105 ['machine']
doc 1106 ['manning']
doc 1107 ['manning']
doc 1108 ['steps']
doc 1109 ['mastering']
doc 1110 ['scikit']
doc 1111 ['clustering']
doc 1112 ['natural']
doc 1113 ['natural']
doc 1114 ['neural']
doc 1115 ['technology']
doc 1116 ['source']
doc 1117 ['digital']
doc 1118 ['fluent']
doc 1119 ['control']
doc 1120 ['series']
doc 1121 ['financial']
doc 1122 ['程序员']
doc 1123 ['logistic']
doc 1124 ['projectmanagement']

doc 1746 ['react']
doc 1747 ['redis']
doc 1748 ['redis']
doc 1749 ['redisbook']
doc 1750 ['redis']
doc 1751 ['little']
doc 1752 ['redis']
doc 1753 ['易运维']
doc 1754 ['riak']
doc 1755 ['oreilly']
doc 1756 ['oreilly']
doc 1757 ['oreilly']
doc 1758 ['oreilly']
doc 1759 ['oreilly']
doc 1760 ['pragmatic']
doc 1761 ['pragmatic']
doc 1762 ['build']
doc 1763 ['build']
doc 1764 ['manning']
doc 1765 ['akka']
doc 1766 ['akka']
doc 1767 ['beginning']
doc 1768 ['functional']
doc 1769 ['manning']
doc 1770 ['oreilly']
doc 1771 ['oreilly']
doc 1772 ['scala']
doc 1773 ['copy']
doc 1774 ['complexity']
doc 1775 ['scala']
doc 1776 ['impatient']
doc 1777 ['high']
doc 1778 ['scala']
doc 1779 ['depth']
doc 1780 ['scalabyexample']
doc 1781 ['影印版']
doc 1782 ['steps']
doc 1783 ['scala']
doc 1784 ['一致性']
doc 1785 ['schemer']
doc 1786 ['screenos']
doc 1787 ['program']
doc 1788 ['oreilly']
doc 1789 ['wiley']
doc 1790 ['application']
doc 1791 ['beginner']
doc 1792 ['selenium']
doc 1793 ['seo']
doc 1794 ['oreilly']
d

# 是否需要按照tf-idf的分数做分类？

从上面的结果看，还并不是很完善，所以更好的办法可能是结合多个模型

In [219]:
filter_file[1568]

'd:/books\\other\\rectangle2.pdf'

In [190]:
import jieba

In [194]:
list(jieba.cut('写给程序员的数据挖掘实践指南'))

['写给', '程序员', '的', '数据挖掘', '实践', '指南']

In [195]:
list(jieba.cut('17_monte_carlo'))

['17', '_', 'monte', '_', 'carlo']